In [2]:
# Dependencies
import torch

import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

SystemError: () method: bad call flags

In [ ]:
# Linear space.
x = torch.linspace(start=0, end=2*np.pi, steps=100, requires_grad=True)  # Or, `.requires_grad_()`
# Check the gradient function that is remembered.
print('x: \n{}'.format(x))
y = torch.sin(input=x)
print('y: \n{}'.format(y))
z = y * 2
print('z: \n{}'.format(z))
w = z + 2
print('w: \n{}'.format(w))
# Composed functions.
print('*'*79)
x = torch.linspace(start=0, end=2*np.pi, steps=100).requires_grad_()
print('x: \n{}'.format(x))
w = torch.sin(x)*2 + 2
print('w: \n{}'.format(w))

In [ ]:
# Branched computations.
x = torch.linspace(start=0, end=2*np.pi, steps=100, requires_grad=True)
y = torch.sin(input=x)
z = y * 2
w = z + 2
u = z + 2
v = z ** 3
print('w: \n{}'.format(w))
print('u: \n{}'.format(u))
print('v: \n{}'.format(v))
print('w == u: {}'.format(
    torch.all(torch.eq(input=w, other=u))
))
print('w is u: {}'.format(w is u))
# Autograd expects the final computation to lead to a scalar BY DEFAULT.
out = (u + v + w).sum()  # Reduces the answer to a scalar of sum of all elements.
print('out: \n{}'.format(out))
print('out.grad_fn: {}'.format(out.grad_fn))  # Checking the grad function.

In [ ]:
# Simple example to visualize gradients.
x = torch.linspace(start=0, end=2*np.pi, steps=20, requires_grad=True)
x_grad_old = x.grad
print('x.grad before `.backward()` call: \n{}'.format(x.grad))
y = torch.sin(input=x)
z = y * 2
w = z + 2
out = w.sum()
out.backward()
x_grads = x.grad
print('x.grad after `.backward()` call: \n{}'.format(x.grad))
x_grad_pred = torch.cos(input=x)*2
plt.style.use('dark_background')
plt.grid(True)
"""
If a tensor requires gradient, then it does not have `.numpy()` method.
Thus, the following will not work--
plt.plot(x.numpy(), x_grads.numpy(), color='green', label='backpropagated gradients', marker='x', alpha=0.8)
plt.scatter(x.numpy(), x_grad_pred.numpy(), color='red', label='predicted backpropagated gradients', marker='o', alpha=0.8)
Instead, for every tensor `X`, we have to call `X.detach().numpy()` to extract the answer to numpy.
"""
plt.plot(x.detach().numpy(), x_grads.detach().numpy(), color='green', label='backpropagated gradients', marker='x', alpha=0.8)
plt.scatter(x.detach().numpy(), x_grad_pred.detach().numpy(), color='red', label='predicted backpropagated gradients', marker='o', alpha=0.8)
plt.legend(loc='best')
plt.show()
"""
After the `.backward()` pass, intermediate tensors will NOT have attached gradients.
"""
print('gradients at y, z, w: {}, {}, {}'.format(y.grad, z.grad, w.grad))  # ALL OF THESE ARE `None`, as stated above!

In [ ]:
"""
Autograd expects the final output to be a scalar in order to call `.backward()` on it.
However, this is NOT a compulsion, which will be covered later.
"""
x = torch.linspace(start=0, end=2*np.pi, steps=20).requires_grad_()
y = torch.sin(input=x)
z1 = y * 2
z2 = y ** 2
z3 = z1 * z2
w = y + z3
out = w.sum()
print('*'*79)  # History for `out`.
print('History of out:')
print('\tgrad function: {}'.format(out.grad_fn))
print('\tnext functions to grad: {}'.format(out.grad_fn.next_functions))
print('*'*79)  # History for `w`.
print('History of w:')
print('\tgrad function: {}'.format(w.grad_fn))
print('\tnext functions to grad: {}'.format(w.grad_fn.next_functions))
print('*'*79)  # History for `z1`.
print('History of z1:')
print('\tgrad function: {}'.format(z1.grad_fn))
print('\tnext functions to grad: {}'.format(z1.grad_fn.next_functions))
print('*'*79)  # History for `z2`.
print('History of z2:')
print('\tgrad function: {}'.format(z2.grad_fn))
print('\tnext functions to grad: {}'.format(z2.grad_fn.next_functions))
print('*'*79)  # History for `z3`.
print('History of z3:')
print('\tgrad function: {}'.format(z3.grad_fn))
print('\tnext functions to grad: {}'.format(z3.grad_fn.next_functions))
print('*'*79)  # History for `y`.
print('History of y:')
print('\tgrad function: {}'.format(y.grad_fn))
print('\tnext functions to grad: {}'.format(y.grad_fn.next_functions))
print('*'*79)  # History for `x`.
print('History of x:')
print('\tgrad function: {}'.format(x.grad_fn))
print('*'*79+'\n'+'*'*79)  # Detailed history for `out`.
print('Detailed history of out:\n')
print('\tgrad: {}'.format(out.grad_fn))
# <SumBackward0 object at <...>> : `out = w.sum()`
print('\tgrad->next: {}'.format(out.grad_fn.next_functions))
# ((<AddBackward0 object at <...>>, 0),) : `w = y + z3`
print('\tgrad->next->next: {}'.format(out.grad_fn.next_functions[0][0].next_functions))
# ((<SinBackward object at <...>>, 0), (<MulBackward0 object at <...>>, 0)) : z3 = z1 * z2